# Interactive minimal working example
In this notebook you are going to run a single simulation, interactively, using the `simulation module` in `dlensalot/ana/config/examples/`, and our goal is to plot the lensing potential estimate.

# Header

In [1]:
import os
from os.path import join as opj
import numpy as np
import healpy as hp

import matplotlib.pyplot as plt

import lenscarf
from lenscarf.run import run

cond4mpi does not exists
could not setup healpix hack. reverting to standard healpy
MSC: using scarf with 2 OMP threads


# Choose analysis

In [2]:
## minimal working example
fn = opj(os.path.dirname(os.path.abspath(lenscarf.__file__)),'ana/config/examples/mwe/conf_mwe_fullsky.py')
print(fn)
ana_mwe = run(config=fn, job_id='QE_lensrec', verbose=True).job

/global/u2/s/sebibel/git/lenscarf/lenscarf/ana/config/examples/mwe/conf_mwe_fullsky.py
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.log - load_configfile() Started: /global/u2/s/sebibel/git/lenscarf/lenscarf/ana/config/examples/mwe/conf_mwe_fullsky.py
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.log - load_configfile() Finished
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.log - store() Started
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.log - load_configfile() Started: /pscratch/sd/s/sebibel/dlensalot/lenscarf/sims_mwe/mwe_lminB200_my_first_dlensalot_analysis/conf_mwe_fullsky.py
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.log - load_configfile() Finished
INFO:: 01-27 08:32:: root.store - config file look the same. Resuming where I left off last time.
INFO:: 01-27 08:32:: root.store - Matching config file found. Resuming where I left off.
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.log - store() Finished
INFO:: 01-27 08:32:: lenscarf.lerepi.core.handler.

# Check configuration
Before running lensing reconstruction, let us look into the configuration file

In [3]:
# ana_mwe.__dict__.keys()
# ana_mwe.sims.__dict__.keys()

In [4]:
# a = ana_mwe.ivfs.sim_lib.get_sim_pmap(0)
# print(a)

## And how does the simulation data look like?

In [48]:
ana_mwe

(3145725,)

In [ ]:
ana_mwe.run()

INFO:: 01-27 08:32:: lenscarf.core.handler.log - run() started
INFO:: 01-27 08:32:: lenscarf.core.handler.run - 0, task calc_phi started
INFO:: 01-27 08:32:: lenscarf.core.handler.run - 0, task calc_meanfield started
INFO:: 01-27 08:32:: lenscarf.core.handler.log - get_sim_qlm(0) started
INFO:: 01-27 08:32:: lenscarf.remapping.__init__ -  Deflection std 4.85e+00 amin
 [00:02:15] collecting bands > 00%
INFO:: 01-27 08:34:: lenscarf.remapping.__init__ -  Deflection std 4.85e+00 amin
 [00:02:15] collecting bands > 00%
generating QU map with 
[   0.           +0.j            0.           +0.j
   -0.           +0.j         ... 1640.72518659-2589.58538379j
 2452.25517539-3214.90999622j -722.70604461+4589.93968269j] [    0.           +0.j             0.           +0.j
     0.           +0.j         ... -5817.11362421 +239.9859219j
 -1349.31377411+2005.10233237j  1520.17075343+1341.51683802j] /pscratch/sd/s/sebibel/dlensalot/lenscarf/sims_mwe/mwe_lminB200_my_first_dlensalot_analysis/ivfs/sim_0